In [1]:
import numpy as np

from num2fixed import decimal_dec_to_bin, dec_to_bin, dec_to_bin_array

In [2]:
# !conda install -y -c conda-forge tensorflow

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.6.11
  latest version: 4.6.14

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/wenqi/anaconda3/envs/py3

  added / updated specs:
    - tensorflow


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    absl-py-0.7.1              |           py36_0         154 KB  conda-forge
    astor-0.7.1                |             py_0          22 KB  conda-forge
    c-ares-1.15.0              |    h1de35cc_1001          81 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    certifi-2019.3.9           |           py36_0         149 KB  conda-forge
    gast-0.2.2                 |             py_0          10 KB  conda-forge
    grpcio-1.16.1              |   py36h044775b_1        

In [10]:
# !conda install -y -c conda-forge keras=2.0.5

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.6.11
  latest version: 4.6.14

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/wenqi/anaconda3/envs/py3

  added / updated specs:
    - keras=2.0.8


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.3.9           |           py36_0         149 KB  conda-forge
    keras-2.0.8                |   py36h39110e4_0         419 KB
    ------------------------------------------------------------
                                           Total:         568 KB

The following NEW packages will be INSTALLED:

  keras              pkgs/main/osx-64::keras-2.0.8-py36h39110e4_0

The following packages will be UPDATED:

  ca-certificates    pkgs/main::ca-certificates-2019.1.23-0 --> conda-forge::ca-certificates-20

In [2]:
# -*- coding: utf-8 -*-
'''VGG16 model for Keras.
# Reference:
- [Very Deep Convolutional Networks for Large-Scale Image Recognition](https://arxiv.org/abs/1409.1556)
'''
from __future__ import print_function

import numpy as np
import warnings

from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import _obtain_input_shape
from keras.engine.topology import get_source_inputs


WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels.h5'
WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'


def VGG16(include_top=True, weights='imagenet',
          input_tensor=None, input_shape=None,
          pooling=None,
          classes=1000):
    """Instantiates the VGG16 architecture.
    Optionally loads weights pre-trained
    on ImageNet. Note that when using TensorFlow,
    for best performance you should set
    `image_data_format="channels_last"` in your Keras config
    at ~/.keras/keras.json.
    The model and the weights are compatible with both
    TensorFlow and Theano. The data format
    convention used by the model is the one
    specified in your Keras config file.
    # Arguments
        include_top: whether to include the 3 fully-connected
            layers at the top of the network.
        weights: one of `None` (random initialization)
            or "imagenet" (pre-training on ImageNet).
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `channels_last` data format)
            or `(3, 224, 244)` (with `channels_first` data format).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 48.
            E.g. `(200, 200, 3)` would be one valid value.
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional layer.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional layer, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
    # Returns
        A Keras model instance.
    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """
    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as imagenet with `include_top`'
                         ' as true, `classes` should be 1000')
    # Determine proper input shape
    input_shape = (224,224,3)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='vgg16')

    # load weights
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('vgg16_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH,
                                    cache_subdir='models')
        else:
            weights_path = get_file('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models')
        model.load_weights(weights_path)
        if K.backend() == 'theano':
            layer_utils.convert_all_kernels_in_model(model)

        if K.image_data_format() == 'channels_first':
            if include_top:
                maxpool = model.get_layer(name='block5_pool')
                shape = maxpool.output_shape[1:]
                dense = model.get_layer(name='fc1')
                layer_utils.convert_dense_weights_data_format(dense, shape, 'channels_first')

            if K.backend() == 'tensorflow':
                warnings.warn('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image data format convention '
                              '(`image_data_format="channels_first"`). '
                              'For best performance, set '
                              '`image_data_format="channels_last"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')
    return model

Using TensorFlow backend.


In [3]:
model = VGG16(include_top=True, weights='imagenet')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [5]:
model.layers

In [8]:
weights = model.get_weights()

In [9]:
len(weights) # 16 weights and 16 biases

32

In [15]:
w1 = np.array(weights[0])
b1 = np.array(weights[1])
print (np.ndarray.flatten(w1).shape)
print (np.ndarray.flatten(b1).shape)

(1728,)
(64,)


In [20]:
for w in weights:
    print(np.max(abs(w)))

0.6714
2.06404
0.289171
1.02715
0.416611
0.365474
0.277376
0.633758
0.544411
0.349496
0.459316
0.274845
0.391537
0.594772
0.313882
0.314845
0.33766
0.182376
0.256236
0.337665
0.190444
0.639749
0.205151
0.917106
0.286997
9.43155
0.0270621
0.855507
0.0326208
1.21546
0.0572556
0.773357


In [22]:
def write_img_bin(weights_np_arr, out_filename='weights.txt', integer_len=8):
    """
    for weights, fixed 8 bits integer, 7 bits decimal
    write the image to a txt file of 0 and 1s
    """
    
    bin_weights = dec_to_bin_array(weights_np_arr, integer_len, 16 - integer_len - 1, decimal_point=False)
    weights_str = "".join(bin_weights)
    
    with open(out_filename, 'w') as f:
        f.write("%s" % weights_str)

## weights ranges:   

layer 1: (0.60851592, -0.67140007)   
layer 2: (0.28917095, -0.21561293)   
layer 3: (0.41661108, -0.33594823)   
layer 4: (0.27737573, -0.18190438)   
layer 5: (0.54441082, -0.18306334)   
layer 6: (0.45931634, -0.17710502)   
layer 7: (0.39153737, -0.17968427)   
layer 8: (0.31388226, -0.12409957)   
layer 9: (0.33766001, -0.10524698)   
layer 10: (0.25623572, -0.12739666)   
layer 11: (0.19044439, -0.11329857)   
layer 12: (0.20515127, -0.13681108)   
layer 13: (0.28699666, -0.092884824)   
layer 14: (0.026629567, -0.027062105)   
layer 15: (0.032620791, -0.021650242)   
layer 16: (0.057255637, -0.035473574) 

However, for bias, we have 9.43155 at max, so we should spend 4 digits on integers, 1 digit on sign and 11 digits on decimals.

In [30]:
all_weights = model.get_weights()

for i in range(16):
    weights = all_weights[2 * i]
    bias = all_weights[2 * i + 1]
    write_img_bin(weights, out_filename='weights_bin/weights_{}.txt'.format(i), integer_len=4)
    write_img_bin(bias, out_filename='weights_bin/bias_{}.txt'.format(i), integer_len=4)

In [29]:
filename = 'weights_{}.txt'.format(0)

with open(filename, 'r') as f:
#     weight_str = str()
    print (len(f.readline()) / 16)
#     f.read(weight_str)÷

1728


# Load Weights

In [29]:
for i in range(16):
    w_name = '../weights_bin/weights_{}.txt'.format(i)
    b_name = '../weights_bin/bias_{}.txt'.format(i)
    
    
    
            
    with open(w_name, 'r') as f:
        w_len = len(f.readline())
        if i == 13 or i == 14:
            param_tab = '\t'
        else:
            param_tab = '\t\t'
        if i >= 7:
            bits_tab = '\t'
        else:
            bits_tab = '\t\t'
        print("weights_{}:\tparameter_nums:\t{}{}bits:\t{}{}bytes:\t{}\t".format(
             i, w_len // 16, param_tab, w_len, bits_tab, w_len // 8))
        
    with open(b_name, 'r') as f:
        b_len = len(f.readline())
        if i < 10: # for indentation only
            bias_tab = '\t'
        else:
            bias_tab = ''
        param_tab = '\t\t'
        print("bias_{}{}:\tparameter_nums:\t{}{}bits:\t{}\t\tbytes:\t{}\t".format(
                i, bias_tab, b_len // 16, param_tab, b_len, b_len // 8))
#         else:
#             print("bias_{}:\tparameter_nums:\t{}\tbits:\t{}\t\tbytes:\t{}\tshorts:\t{}\t".format(
#                  i, b_len // 16, b_len, b_len // 8, b_len // 16))

weights_0:	parameter_nums:	1728		bits:	27648		bytes:	3456	
bias_0	:	parameter_nums:	64		bits:	1024		bytes:	128	
weights_1:	parameter_nums:	36864		bits:	589824		bytes:	73728	
bias_1	:	parameter_nums:	64		bits:	1024		bytes:	128	
weights_2:	parameter_nums:	73728		bits:	1179648		bytes:	147456	
bias_2	:	parameter_nums:	128		bits:	2048		bytes:	256	
weights_3:	parameter_nums:	147456		bits:	2359296		bytes:	294912	
bias_3	:	parameter_nums:	128		bits:	2048		bytes:	256	
weights_4:	parameter_nums:	294912		bits:	4718592		bytes:	589824	
bias_4	:	parameter_nums:	256		bits:	4096		bytes:	512	
weights_5:	parameter_nums:	589824		bits:	9437184		bytes:	1179648	
bias_5	:	parameter_nums:	256		bits:	4096		bytes:	512	
weights_6:	parameter_nums:	589824		bits:	9437184		bytes:	1179648	
bias_6	:	parameter_nums:	256		bits:	4096		bytes:	512	
weights_7:	parameter_nums:	1179648		bits:	18874368	bytes:	2359296	
bias_7	:	parameter_nums:	512		bits:	8192		bytes:	1024	
weights_8:	parameter_nums:	2359296		bits:	37748736	byte